# Introduction

This notebook is intended for presenting processes and tasks done with the use of Python. Below are objectives to be completed through this notebook:
* To  use a dataset consisting of politics-related tweets written in Filipino language for training the model;
* To train the three (3) selected models and apply hyperparameter tuning and optimization using random search optimization and experimentation; (*This was done in separate, duplicate notebooks*)
* To compare trained models for the performance between transformer-based models and the baseline model, Multinomial Naïve Bayes, and find the best performing model.

The notebook is split into the following subsections:
1. *Installation and Imports* - This section consists of installing dependencies and importing necessary libraries to run the notebook and the models, data, and techniques to be used.
2. *Environment Setup and Data Loading* - This section involves loading the dataset, splitting into training and testing, and preparation for modeling.
3. *Tokenization* - This section involves the process of tokenizing the data by loading the tokenizer and defining a tokenization function.
4. *Model and Metrics* - This section involves defining functions for model initialization and computing metrics.
5. *Training Arguments and Training Setup* - This section involves defining the arguments of the hyperparameter value ranges to be used for optimization and searching of the best set of values.
6. *Traditional Model Setup and Training* - This section handles the setup of  the Multinomial Naive Bayes model to be compared to the BERT models.
________________________________________________________________________________

**Dataset Information**

The dataset, obtained from HuggingFace as a secondary source, contains data collected by PhD student Jan Christian Blaise Cruz. The data consists of social media content (specifically tweets) from the 2016 presidential election in the Philippines. The dataset has 2 columns, one for the extracted text and the other being a label indicating whether the text contains hate speech or not (1 or 0 respectively). According to Cruz, the dataset was released having 4232 samples each for validation and testing and contains about 10k rows/samples for the training split .


# Installation and Imports

This sections setups up the environment by first installing dependencies and then, importing the necessary libraries that will be used for model training, dataset handling, and hyperparameter tuning.

The code cell below installs the latest verions of the required libraries. It uses the *pip* shell command to install *transformers*, *datasets*, *accelerate*, *ray[tune]*, and *optuna* hich are used for NLP models, loading datsets, and performing hyperparameter tuning. The *-U* at the end of the line indicates that the latest version of each package must be installed.

In [ ]:
# Install dependencies
!pip install transformers datasets accelerate ray[tune] optuna -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.3/108.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 MB 12.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: click
    Found existing installation: click 8.3.0
    Uninstalling click-8.3.0:
      Successfully uninstalled click-8.3.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0


The code cell below involves importing the necessary libraries for performing hyperparameter tuning using automated optimization techniques. It helps prepare the environment by adding tools that help with handling the data, defining the model, configuring training, and evaluating performance. The following libraries are imported:

1. *torch* - This is the library by PyTorch that handles tensor operations and GPU acceleration.
2. *os* - This library helps by giving access to operating system functions like file handling and environment variables.
3. *numpy* - This was used for numerical operations, especially predictions and metrics.
4. *pandas* - This is mainly used for reading and manipulating tabular data. In this case, it was used for reading the csv file.
5. *accuracy_score, f1_score* - These are the evaluation metrics used for checking the performance of the model.
6. *files* - This is by Google Colab that allows file uploads from the local machine to the Colab environment.
7. *Dataset* - This is the class by Hugging Face that wraps pandas DataFrames as model-ready datasets.
8. *AutoTokenizer* - Hugging face tool for automaticallly loading the right tokenizer for the model.
9. *AutoModelForSequenceClassification* - Another hugging face tool for loading model for text classification.
10. *Trainer* - Hugging face tool for simplifying training and evaluation.
11. *TrainingArguments* - Library for configuring the training settings.
12. *set_seed* - Library that ensures the reproducibility by fixing random seeds.

In [ ]:
# Import necessary libraries
import torch
import os
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score
from google.colab import files
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    set_seed
)

# Environment Setup and Data Loading

This section sets up the environment by performing steps such as setting seed to ensure reproducibility, detecting the presence of a GPU, uploading of dataset, splitting the data into training and evaluation dataset, and converting it into a suitable format. This prepares the data for tokenization and model training.

The code cell below ensures that the results are consistent every time the notebook is run. This is done by setting a fixed random seed using *set_seed()* with the value of *42*.

In [ ]:
# Set seed for reproducibility
set_seed(42)

The code cell below aims to detect whether the system has a GPU or not and sets the appropriate device for training the model. It makes sure that the model will use the most efficient hardware available. Using a higher performance hardware (in this case, using GPU instead of CPU) affects the amount of time it takes to train the model and the amount of memory it uses.

*if torch.cuda.is_available()* check whether a GPU is available. If it is available, it assigns the GPU as the computation device, stores it in the *device* variable and prints the name of the GPU to the user. If it is not available, it sets the CPU as the computation device and stores it in the *device* variable and informs the user that GPU is not available for use.

In [ ]:
# Check device
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")

The code cell below uploads the dataset. In this case, the dataset is in CSV format and has the file name *hate_speech*.

To upload the file from the local machine to the Colab environment, it uses *files.upload()* and stores the file in the *uploaded* variable. It, then, reads the CSV file using *pd.read_csv()* by pandas and stores it in the *df* variable. Finally, it ensures that it keeps only the relevant columns (those with the name of *text* and *label*). Since the dataset contains only these two columns, it keeps all of the columns of the dataset.

In [ ]:
# Upload and load daataset
uploaded = files.upload()
df = pd.read_csv("hate_speech.csv", encoding="latin-1")[["text", "label"]]

Saving hate_speech.csv to hate_speech.csv


The code cell below splits the data into two parts: 1000 samples for training, and 200 samples for evaluation. As seen in the code cell below, it uses *.iloc[]* in identifying the rows to be added on the indicated set.

In [ ]:
# Split into train and eval
train_data = df.iloc[:1000].reset_index(drop=True)
eval_data = df.iloc[1000:1200].reset_index(drop=True)

The code cell below converts data into a compatible format. As seen below, te pandas DataFrame is first converted into Hugging Face dataset. This uses the *Dataset.from_pandas()* method on the *train_data* and *eval_data* and stores it on the *train_dataset* and *eval_dataset* variables respectively.

In [ ]:
# Convert pandas DataFrames to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_data)
eval_dataset = Dataset.from_pandas(eval_data)

# Tokenization

This section focuses on the process of turning the raw text into a numerical format that the model can understand. In this section, the tokenizer is being loaded and a tokenization function was defined and applied on the data.

The code cell below initializes the tokenizer to be used for converting the Tagalog text into tokens.

The model name which is *jcblaise/roberta-tagalog-base* is stored in the variable *MODEL_NAME*. After which, the tokenizer is loaded using *AutoTokenizer.from_pretrained()* with the model name as its argument and stores it in the *tokenizer* variable.

In [ ]:
# Load tokenizer
MODEL_NAME = "jcblaise/bert-tagalog-base-cased-WWM"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/55.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

The code cell below focuses on creating a function that tokenizes the text in the dataset. As seen on the first line of the code cell, the function is named *tokenize_function* with *examples* as its argument. It uses the function *tokenizer()* which applies truncation and padding to ensure that the input length is consistent. This ensures that the text from the dataset undergoes consistent preprocessing prior to training and evaluation.

As seen in the code cell below, the padding has or the max length has been set to *256* which is the same value that got the best results in the results of one of my members.

In [ ]:
# Tokenization function
def tokenize_function(examples):
        return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=256)

The code cell below applies the tokenization function by applying *map()* on both the *train_dataset* and *eval_dataset* and storing them on the variables *tokenized_train* and *tokenized_eval* respectively.

In [ ]:
# Apply tokenization
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_eval = eval_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

The code cell below renames the label column to labels to ensure that it is compatible with the Trainer class during training and evaluation. As seen in the code cell below, it uses the *rename_column()* function to match the expected format of Hugging Face's Trainer.

The code cell below also prepares the tokenized data for PyTorch-based training. It also ensures proper format of the input for training and evaluation. As seen in the code cell below, the *set_format("torch",...)* part ensures that the dataset columns are converted into PyTorch tensors.

In [ ]:
# Rename 'label' to 'labels' and set format to PyTorch tensors
tokenized_train = tokenized_train.rename_column("label", "labels")
tokenized_eval = tokenized_eval.rename_column("label", "labels")

tokenized_train.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_eval.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# Model and Metrics

This section involves the definition of functions for modelling and computing the metrics to ensure consistent model setup and gauge the performance of the models.

The code cell below initializes a fresh model for every grid search run. It does this by creating a new instance of the model for each hyperparameter trial. As seen in the cell below, the function is defined as *model_init()*.

What this function does is that it loads the model (in this case, the RoBERTA Tagalog base model). In the loading of the model, it ensures that the model has been specified for binary classification using *num_labels=2*. The dropout rate has also been declared with the value of *0.2* and assigned to *hidden_dropout_prob* and *attention_probs_dropout_prob*. The value was based on the results of one of my members which indicated 0.2 as the best performing value for the dropout rate.

In [ ]:
# Function to initialize a fresh model for each grid search run
def model_init():
  return AutoModelForSequenceClassification.from_pretrained(
      MODEL_NAME,
      num_labels=2,
      hidden_dropout_prob=0.1,
      attention_probs_dropout_prob=0.1
  ).to(device)

The code cell below focuses on defining the metrics to be used for evaluating the performance of the model. As seen in the code cell below, a function named *compute_metrics* is defined. It basically calculates the accuracy score and the F1 score for binary classification using the functions *accuracy_score()* and *f1_score()* and stores them in the variables *acc* and *f1* respectively. Both of these metrics are then returned as a dictionary.

In [ ]:
# Metrics function
def compute_metrics(p):
    # Uses F1-Score as the primary metric for comparison
    preds = np.argmax(p.predictions, axis=1)
    acc = accuracy_score(p.label_ids, preds)
    f1 = f1_score(p.label_ids, preds, average="binary")
    return {"accuracy": acc, "f1": f1}

# Random Search

The code cell below declares the **tune_hp** defined function. The function creates the space where the different hyperparameter values will be placed in. The **trial** object samples different declared values for key parameters that influence training behavior and performance. Once the function is used, each trial will make use of random hyperparameter values from the given range (such as in line 17, where it will randomly use logging_step values between 50 and 200). The function will then return the value used in a dictionary which will be passed to the model's training loop, allowing logging as Optuna evaluates and compares different values for each hyperparameter.

In [ ]:
def tune_hp(trial):
    """
    This function defines the hyperparameter space to be explored.
    The `trial` object allows us to suggest different values.
    """
    # The grid uses the trial.suggest_categorical and trial.suggest_float methods
    # from the Optuna backend, which is highly efficient.

    # 1. Learning Rate (Critical for performance)
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True)

    # 2. Batch Size (Affects VRAM and stability)
    train_batch_size = trial.suggest_float("per_device_train_batch_size", 16, 32, log=True)
    eval_batch_size = trial.suggest_float("per_device_eval_batch_size", 16, 32, log=True)

    # 3. Logging Steps
    logging_steps = trial.suggest_float("logging_steps", 50, 200, log=True)

    # 4. Warmup Steps
    warmup_steps = trial.suggest_float("warmup_steps", 250, 1000, log=True)

    # 5. Weight Decay
    weight_decay = trial.suggest_float("weight_decay", 0, 0.05)

    # 6. FP16
    fp16 = trial.suggest_categorical("fp16", [True, False])

    return {
        "learning_rate": learning_rate,
        "per_device_train_batch_size": train_batch_size,
        "per_device_eval_batch_size": eval_batch_size,
        "logging_steps": logging_steps,
        "warmup_steps": warmup_steps,
        "weight_decay": weight_decay,
        "fp16": fp16,
    }

# Training Arguments and Trainer Setup

This section configures how the model was trained and evaluated. It involves defining fixed training settings for the hyperparameters and the initialization of the Trainer.

The code cell below is responsible for configuring how the model will be trained, evaluated, and logged. As seen in the code cell below, it consists of multiple hyperparameters. For the purposes of this exercise, the evaluation settings were not changed. Parameters such as *fp16*, *num_train_epochs*, *warmup_steps*, and *weight_decay* have fixed values which were all based on the results of the previous exercise.

In [ ]:
# Training arguments (fixed for all runs)
training_args = TrainingArguments(
    output_dir="./grid_search_results",
    # Evaluation settings (fixed)
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1", # Optimize for F1-Score
    report_to="none",
)

The code cell below initializes the Hugging Face Trainer with all the necessary components including the model, training and testing dataset, matrics, and tokenizer. As seen in the code ccell below the Hugging Face Trainer is stored in the *trainer* variable with arguments such as *model*, *args*, *train_dataset*, *eval_dataset*, *compute_metrics*, and *tokenizer*. This prepares the training engine that will fine-tune the model and evaluate its performance.

In [ ]:
# Initialize the Trainer
trainer = Trainer(
    model_init=model_init, # We pass the function, not the object, for fresh initialization
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

/tmp/ipython-input-1532091534.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at jcblaise/bert-tagalog-base-cased-WWM and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

The code cell below initiates the hyperparameter search using the Random Search technique. With the use of **Optuna** as a backend and Hugging Face's **trainer.hyperparameter_search()** API, an automated search for the best set of hyperparameters which targets the best valued F1-Score is conducted. The set of hyperparameters are sourced from the declared value range in the **tune_hp** space, while the use of Optuna as an argument ensures efficient sampling and pruning. The **direction_maximize** configuration is what instructs Optuna to select the set of values that produce the best F1-score. The result, stored in **best_trial**, contains the best-performing hyperparameters discovered during the search.

Pruned trials are those that fail two produce optimal F1-Scores. Additionally, a count of 18 trials was set to be done (0-17).

In [ ]:
# Execution of Random Search
# We use Optuna backend for efficient searching. The 'hp_space' provides the search definition.
print("\n--- Starting Random Search (Total Runs: 18) ---")
print("Optimizing for 'f1' score...")

best_trial = trainer.hyperparameter_search(
    # We use 'Optuna' as the backend for the hyperparameter search
    backend="optuna",
    # Pass the function that defines the search space
    hp_space=tune_hp,
    # Maximize the F1 score (higher is better)
    direction="maximize",
    # Set the total number of experiments to run
    n_trials=18,
)

[I 2025-11-17 09:56:45,686] A new study created in memory with name: no-name-477ce9d4-7ee1-408c-83b5-907b1ac09bca



--- Starting Random Search (Total Runs: 5) ---
Optimizing for 'f1' score...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at jcblaise/bert-tagalog-base-cased-WWM and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.692285,0.540000,0.402597
2,No log,0.647925,0.630000,0.614583
3,0.652600,0.599203,0.705000,0.748936


[I 2025-11-17 09:59:34,171] Trial 0 finished with value: 1.4539361702127658 and parameters: {'learning_rate': 3.119814329835857e-05, 'per_device_train_batch_size': 22.216040575545613, 'per_device_eval_batch_size': 25.54089700860461, 'logging_steps': 93.8923411579136, 'warmup_steps': 415.64319475650296, 'weight_decay': 0.0018982749277018096, 'fp16': True}. Best is trial 0 with value: 1.4539361702127658.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at jcblaise/bert-tagalog-base-cased-WWM and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

[I 2025-11-17 10:55:23,691] Trial 15 pruned. 
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at jcblaise/bert-tagalog-base-cased-WWM and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.692264,0.550000,0.423077
2,No log,0.650218,0.630000,0.614583
3,0.661000,0.617444,0.680000,0.711712


[I 2025-11-17 11:00:59,586] Trial 16 finished with value: 1.3917117117117117 and parameters: {'learning_rate': 2.0237348021496265e-05, 'per_device_train_batch_size': 25.25974625169003, 'per_device_eval_batch_size': 22.999708169066125, 'logging_steps': 91.31820260954274, 'warmup_steps': 327.86469718156144, 'weight_decay': 0.027035705529987, 'fp16': True}. Best is trial 0 with value: 1.4539361702127658.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at jcblaise/bert-tagalog-base-cased-WWM and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.692051,0.540000,0.402597
2,0.665800,0.647930,0.630000,0.614583
3,0.665800,0.601161,0.700000,0.743590


[I 2025-11-17 11:04:24,539] Trial 17 pruned. 


Below prints the set of hyperparameters that yielded the best F1-score. An if-else statement was done to check whether there is an actual best trial from the result, otherwise it would print a message indicating a failure of searching or when there is no best trial. The hyperparameters of the best trial is listed, presenting values that were taken from the provided range of values.

In [ ]:
print("\n--- Random Search Complete ---")
print("\nBEST HYPERPARAMETERS FOUND:")

# Extract and print the best configuration
if best_trial:
    print(best_trial)
    best_hps = best_trial.hyperparameters
    print("\nBest Hyperparameters:")
    for key, value in best_hps.items():
        print(f"  {key}: {value}")
else:
    print("Search failed or no best trial found.")

print("\nTo run the final model, use the best_hps found in a new TrainingArguments instance.")


--- Random Search Complete ---

BEST HYPERPARAMETERS FOUND:
BestRun(run_id='0', objective=1.4539361702127658, hyperparameters={'learning_rate': 3.119814329835857e-05, 'per_device_train_batch_size': 22.216040575545613, 'per_device_eval_batch_size': 25.54089700860461, 'logging_steps': 93.8923411579136, 'warmup_steps': 415.64319475650296, 'weight_decay': 0.0018982749277018096, 'fp16': True}, run_summary=None)

Best Hyperparameters:
  learning_rate: 3.119814329835857e-05
  per_device_train_batch_size: 22.216040575545613
  per_device_eval_batch_size: 25.54089700860461
  logging_steps: 93.8923411579136
  warmup_steps: 415.64319475650296
  weight_decay: 0.0018982749277018096
  fp16: True

To run the final model, use the best_hps found in a new TrainingArguments instance.
